In [1]:
from hcmus.utils import data_utils

splits = data_utils.get_data_splits()
datasets = data_utils.get_image_datasets_v2(splits, random_margin=0)

2025-07-07 10:40:06.861 | INFO     | hcmus.core.appconfig:<module>:7 - Load DotEnv: True
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2025-07-07 10:40:07.987 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:148 - Page size is too large, only 3443 tasks available.
2025-07-07 10:40:07.988 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:152 - New `page_to` applied: 35
Loading tasks: 100%|██████████| 35/35 [00:10<00:00,  3.24it/s]
2025-07-07 10:40:24.872 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:148 - Page size is too large, only 435 tasks available.
2025-07-07 10:40:24.873 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:152 - New `page_to` applied: 5
Loading tasks: 100%|██████████| 5/5 [00:03<00:00,  1

In [6]:
from loguru import logger
from torchvision import transforms as T
from hcmus.models.backbone import CLIPBackbone
from hcmus.models.backbone import DinoBackbone

In [3]:
device = "mps"
backbone_list = [
    (CLIPBackbone, {"backbone_name": "ViT-B/32"}),
    (CLIPBackbone, {"backbone_name": "ViT-B/16"}),
    (DinoBackbone, {"model_id": "facebook/dinov2-small"}),
    (DinoBackbone, {"model_id": "facebook/dinov2-base"}),
    (DinoBackbone, {"model_id": "facebook/dino-vitb8"}),
    (DinoBackbone, {"model_id": "facebook/dino-vits8"}),
    (DinoBackbone, {"model_id": "facebook/dino-vits16"}),
    (DinoBackbone, {"model_id": "facebook/dino-vitb16"}),
]

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

In [14]:
import faiss
from tqdm import tqdm

def build_index(backbone_config):
    cls, params = backbone_config
    backbone = cls(**params)
    output_dim = backbone.output_dim
    logger.info(f"Backbone: {cls.__name__}, params={params}")
    logger.info(f"Output dim: {backbone.output_dim}")
    index = faiss.IndexHNSWFlat(output_dim, 256)
    labels = []
    for item in tqdm(datasets["train"], desc="Building index..."):
        image, label, metadata = item
        tensor = transform(image)
        feature = backbone.forward(tensor)
        feature = feature.detach().numpy().astype("float32")
        labels.append((label, metadata["label_str"]))
        index.add(feature)
    return backbone, index, labels

In [33]:
def evaluate(backbone, index, labels, split_name: str):
    precision_1 = 0
    precision_3 = 0
    precision_5 = 0

    total = len(datasets[split_name])
    for item in tqdm(datasets[split_name], desc="Evaluating dataset=val..."):
        image, label, metadata = item
        tensor = transform(image)
        feature = backbone.forward(tensor)
        feature = feature.detach().numpy().astype("float32")
        _, I = index.search(feature, k=5)
        preds = [labels[i][0] for i in I[0]]
        precision_1 += preds[0] == label
        precision_3 += label in preds[:2]
        precision_5 += label in preds[:4]

    logger.info(f"Precision@1 dataset={split_name}: {precision_1 / total}")
    logger.info(f"Precision@3 dataset={split_name}: {precision_3 / total}")
    logger.info(f"Precision@5 dataset={split_name}: {precision_5 / total}")


In [34]:
for config in backbone_list:
    backbone, index, labels = build_index(config)
    evaluate(backbone, index, labels, "val")
    evaluate(backbone, index, labels, "test")

2025-07-07 11:13:50.336 | INFO     | __main__:build_index:8 - Backbone: CLIPBackbone, params={'backbone_name': 'ViT-B/32'}
2025-07-07 11:13:50.337 | INFO     | __main__:build_index:9 - Output dim: 512
Evaluating dataset=val...: 100%|██████████| 2824/2824 [02:03<00:00, 22.82it/s]
2025-07-07 11:17:19.116 | INFO     | __main__:evaluate:18 - Precision@1 dataset=val: 0.5722379603399433
2025-07-07 11:17:19.116 | INFO     | __main__:evaluate:19 - Precision@3 dataset=val: 0.6388101983002833
2025-07-07 11:17:19.116 | INFO     | __main__:evaluate:20 - Precision@5 dataset=val: 0.6855524079320113
Evaluating dataset=val...: 100%|██████████| 5836/5836 [04:24<00:00, 22.10it/s]
2025-07-07 11:21:43.168 | INFO     | __main__:evaluate:18 - Precision@1 dataset=test: 0.4885195339273475
2025-07-07 11:21:43.168 | INFO     | __main__:evaluate:19 - Precision@3 dataset=test: 0.569396847155586
2025-07-07 11:21:43.169 | INFO     | __main__:evaluate:20 - Precision@5 dataset=test: 0.6271418779986292
/Volumes/Cucumb

In [35]:
backbone, index, labels = build_index((DinoBackbone, {"model_id": "facebook/dinov2-large"}))
evaluate(backbone, index, labels, "val")
evaluate(backbone, index, labels, "test")

2025-07-07 18:28:05.669 | INFO     | __main__:build_index:8 - Backbone: DinoBackbone, params={'model_id': 'facebook/dinov2-large'}
2025-07-07 18:28:05.670 | INFO     | __main__:build_index:9 - Output dim: 1024
Evaluating dataset=val...: 100%|██████████| 2824/2824 [11:03<00:00,  4.25it/s]
2025-07-07 18:48:51.888 | INFO     | __main__:evaluate:18 - Precision@1 dataset=val: 0.5995042492917847
2025-07-07 18:48:51.888 | INFO     | __main__:evaluate:19 - Precision@3 dataset=val: 0.6724504249291785
2025-07-07 18:48:51.888 | INFO     | __main__:evaluate:20 - Precision@5 dataset=val: 0.740084985835694
Evaluating dataset=val...: 100%|██████████| 5836/5836 [21:44<00:00,  4.47it/s]
2025-07-07 19:10:36.796 | INFO     | __main__:evaluate:18 - Precision@1 dataset=test: 0.4859492803289925
2025-07-07 19:10:36.796 | INFO     | __main__:evaluate:19 - Precision@3 dataset=test: 0.5755654557916381
2025-07-07 19:10:36.797 | INFO     | __main__:evaluate:20 - Precision@5 dataset=test: 0.6692940370116518
